In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,Flatten,Input,GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d ismailpromus/skin-diseases-image-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip skin-diseases-image-dataset.zip

unzip:  cannot find or open skin-diseases-image-dataset.zip, skin-diseases-image-dataset.zip.zip or skin-diseases-image-dataset.zip.ZIP.


In [ ]:

train_path='/content/gdrive/MyDrive/skin-image-dataset/train'
test_path='/content/gdrive/MyDrive/skin-image-dataset/test'
new_dir=[train_path,test_path]
for dir in new_dir:
  try:
    os.mkdir(dir)
  except:
    shutil.rmtree(dir)
    os.mkdir(dir)

FileNotFoundError: ignored

In [ ]:
file_src='/content/IMG_CLASSES'
folder_list=os.listdir(file_src)

FileNotFoundError: ignored

In [ ]:
for folder in folder_list:
  for dir in new_dir:
    try:
      os.mkdir(os.path.join(dir,folder))
    except:
      shutil.rmtree(os.path.join(dir,folder))
      os.mkdir(os.path.join(dir,folder))

In [ ]:
split_rate=0.8
for folder in folder_list:

  train_dir=os.path.join(train_path,folder)
  test_dir=os.path.join(test_path,folder)
  src_folder=os.path.join(file_src,folder)
  src_file=os.listdir(os.path.join(file_src,folder))
  random.seed(44)
  random.shuffle(src_file)
  split_num=int(len(src_file)*0.8)
  for img_file in src_file[:split_num]:
    src=os.path.join(src_folder,img_file)
    dest=os.path.join(train_dir,img_file)
    shutil.copyfile(src,dest)
  for img_file in src_file[split_num:]:
    src=os.path.join(src_folder,img_file)
    dest=os.path.join(test_dir,img_file)
    shutil.copyfile(src,dest)

In [ ]:
img_size=(224,224)
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   rotation_range=0.2)
train_gen = train_datagen.flow_from_directory(directory=train_path,
                                              batch_size=64,
                                              class_mode='categorical',
                                              target_size=img_size)
valid_datagen=ImageDataGenerator(rescale=1/255.0)
valid_gen = valid_datagen.flow_from_directory(directory=test_path,
                                              batch_size=64,
                                              class_mode='categorical',
                                              target_size=img_size)

In [ ]:
img_size=224
resnet50_model=ResNet50(include_top=False,input_shape=(img_size,img_size,3),
                        weights='imagenet',classes=10)

resnet50_x=Flatten()(resnet50_model.output)
resnet50_x=Dense(1024,activation='relu')(resnet50_x)
resnet50_x=Dropout(0.5)(resnet50_x)
resnet50_x=Dense(10,activation='softmax')(resnet50_x)
resnet50_x_final_model=Model(inputs=resnet50_model.input,outputs=resnet50_x)
resnet50_x_final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                               loss='categorical_crossentropy',
                               metrics=['accuracy'])

EPOCHS=60
resnet_es=EarlyStopping(monitor='val_loss',mode='min',patience=4)
CALLBACKS=[resnet_es]
resnet50_history=resnet50_x_final_model.fit(train_gen,epochs=EPOCHS,
                                            validation_data=valid_gen,
                                            callbacks=CALLBACKS,
                                            verbose=1)



In [ ]:
def plt_hist(history):
  plt.figure(figsize=(10,5))
  plt.subplot(1,2,1)
  plt.plot(history.history['accuracy'],label='accuracy')
  plt.plot(history.history['val_accuracy'],label='val_accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend(loc='upper left')

  plt.subplot(1,2,2)
  plt.plot(history.history['loss'],label='loss')
  plt.plot(history.history['val_loss'],label='val_loss')
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.legend(loc='upper left')

plt.tight_layout()

In [ ]:
plt_hist(resnet50_history)

In [ ]:
effnetb7_base = EfficientNetB7(weights="imagenet", include_top=False, input_shape=(224,224,3))

In [ ]:
model = Sequential()
model.add(effnetb7_base)
model.add(GlobalMaxPooling2D())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                               loss='categorical_crossentropy',
                               metrics=['accuracy'])

EPOCHS=60
es_effnet=EarlyStopping(monitor='val_loss',mode='min',patience=4)
filepath_effnet="best_weights_effnet.hdf5"
mc_effnet= ModelCheckpoint(filepath_effnet, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
CALLBACKS=[es_effnet,mc_effnet]
model_history=model.fit(train_gen,epochs=EPOCHS,
                                            validation_data=valid_gen,
                                            callbacks=CALLBACKS)
